In [1]:
import re
from aocd import data
from dataclasses import dataclass
from itertools import count
from more_itertools import batched


In [2]:
@dataclass
class Range:
    dest_start: int
    source_start: int
    range_length: int

    def __getitem__(self, source):
        if source in range(self.source_start, self.source_start + self.range_length):
            offset = source - self.source_start
            return self.dest_start + offset
        return None
    
    def source(self, dest):
        if dest in range(self.dest_start, self.dest_start + self.range_length):
            offset = dest - self.dest_start
            return self.source_start + offset
        return None

@dataclass
class Map:
    ranges: list[Range]

    def __getitem__(self, source):
        for r in self.ranges:
            if dest := r[source]:
                return dest
        return source
    
    def source(self, dest):
        for r in self.ranges:
            if source := r.source(dest):
                return source
            if r.dest_start > dest:
                break
        return dest


In [4]:
def parse_line(l):
    return [int(n) for n in re.findall(r'\d+', l)]

def parse_map(m):
    ranges = [Range(*parse_line(l)) for l in m.splitlines()[1:]]
    return Map(sorted(ranges, key=lambda r: r.dest_start))


In [5]:
almanac     = data.split('\n\n')
seeds       = parse_line(almanac[0])
soil        = parse_map(almanac[1])
fertilizer  = parse_map(almanac[2])
water       = parse_map(almanac[3])
light       = parse_map(almanac[4])
temperature = parse_map(almanac[5])
humidity    = parse_map(almanac[6])
location    = parse_map(almanac[7])

def location_number(seed):
    return location[humidity[temperature[light[water[fertilizer[soil[seed]]]]]]]


In [6]:
min(location_number(seed) for seed in seeds)


600279879

In [7]:
def seed_number(loc):
    return soil.source(fertilizer.source(water.source(light.source(temperature.source(humidity.source(location.source(loc)))))))

# for loc in count():
#    if seed_number(loc) in seeds:
#        print(loc)
#        break


In [8]:
seed_ranges = sorted([range(start, start+length) for start, length in batched(seeds, 2)], key=lambda r:r.start)
def min_seed():
    for loc in count():
        seed = seed_number(loc)
        for r in seed_ranges:
            if seed in r:
               return loc
            if r.stop > seed:
                break
min_seed()


20191102